In [12]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
import sqlite3

dbpath = "./thumbnail.db" 

conn = sqlite3.connect(dbpath)
cur = conn.cursor() 

In [14]:
df = pd.read_sql_query("SELECT * FROM thumbnail", conn) 

In [15]:
data = df[["title"]]

In [34]:
data['title'][10]

' ROTC 음악회  테너 김호중  베이스 권순동  향수  정지용 시 김희갑 곡 '

In [17]:
tfidf = TfidfVectorizer(stop_words='english')

In [18]:
tfidf_matrix = tfidf.fit_transform(data['title'])
print(tfidf_matrix.shape)

(1752, 8095)


In [20]:
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_matrix.shape

(1752, 1752)

In [21]:
np.round(cosine_matrix, 4)

array([[1.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 1.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 1.   , ..., 0.   , 0.   , 0.   ],
       ...,
       [0.   , 0.   , 0.   , ..., 1.   , 0.262, 0.   ],
       [0.   , 0.   , 0.   , ..., 0.262, 1.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 1.   ]])

In [22]:
# movie title와 id를 매핑할 dictionary를 생성해줍니다. 
title2idx = {}
for i, c in enumerate(data['title']): 
    title2idx[i] = c

# id와 movie title를 매핑할 dictionary를 생성해줍니다. 
idx2title = {}
for i, c in title2idx.items(): 
    idx2title[c] = i

In [37]:
idx = idx2title['김경재  박근혜 구하기   삼성 출연금 8천억  노무현이 해먹었다']
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:10] # 상위 10개의 인덱스와 유사도를 추출 

[(353, 0.14752676803650316),
 (35, 0.12313417641487309),
 (130, 0.10843284505088231),
 (14, 0.10780769146882162),
 (127, 0.10048776968130432),
 (413, 0.09514882383813604),
 (409, 0.09508545613946681),
 (393, 0.09440321417467501),
 (394, 0.09359587954945964),
 (200, 0.09266696510446892)]

In [38]:
sim_scores = [(title2idx[i], score) for i, score in sim_scores[0:10]]
sim_scores

[('미국도 박근혜 결단 지지  이명박 박근혜 잡으려다가 문재인 잡는 통일부', 0.14752676803650316),
 (' 김경재 격정토로  김대중은 우파인 한화갑과 나를   ', 0.12313417641487309),
 ('박근혜   하야하라   퇴진하라  듣고 난 후 표정', 0.10843284505088231),
 ('김경재 총재   탄핵광풍 배후 JTBC 회장 홍석현을 만났더니   ', 0.10780769146882162),
 ('박근혜 서문시장 방문  믿기지 않는 장면', 0.10048776968130432),
 ('박근혜 대통령 탄핵은 답 정 너  정규재 뉴스칼럼  3월 8일 ', 0.09514882383813604),
 ('김경재 前 총재   위기의 대한민국  해결책 있다   정규재 주필  펜앤초대석   7월 9일 ', 0.09508545613946681),
 ('정규재 특별대담 박근혜 前대통령의 육성 반격 2017년1월25일 ', 0.09440321417467501),
 ('고영태 녹음파일2  박근혜 죽이고   정규재 칼럼  2017년2월16일 ', 0.09359587954945964),
 ('서울의소리 김을동 응징취재   박근혜 밑에서 권력을 누리더니   왜   ', 0.09266696510446892)]